In [3]:
import pandas as pd
import numpy as np
import requests
import random

from geopy.geocoders import Nominatim
from pandas import json_normalize  #--use this one!!
import folium
import wget
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.pyplot as plt


#### set up parameters for call later (I'm not showing the client_id)

In [236]:
client_id = "042PAXPIJP1BTANAVAMVMPAX55ABB520NRDH1OXR3MDK5P2Q"
client_srt = "STE5MQZGMNGCVYWLBX22V5CZAJU2ZVM2HSOX1TCZYHCR1HRI"
version = "20200527"
limit = 100
radius = 500

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode("Toronto, Canada")
print(location)
can_lat = location.latitude
can_lng = location.longitude
print(latitude, longitude)


Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
43.6534817 -79.3839347


#### read html use pd.read_html, then check if all neighborhoods have borough

In [70]:
#---read html
html_string = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df_tot = pd.read_html(html_string)[0].rename(columns = {"Postal Code":"PostalCode"})   
#--after read the html, it is a list, and [0] of that list is the DF I need


df_tot[df_tot["Borough"] != "Not assigned"]["Neighborhood"].isnull().unique() #--all not assigned have a neighborhood!

df_tot2 = df_tot[df_tot["Borough"] != "Not assigned"].sort_values(by = "PostalCode", ascending = True)

df_tot2.shape  #-----(103, 3)
df_tot2.head()

,PostalCode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


#### use csv to get postal codes then quality check the data

In [71]:
#--get lat and lng for each postal code

df_toronto = df_tot2.join(pd.read_csv("http://cocl.us/Geospatial_data").set_index("Postal Code"), on = "PostalCode")

df_toronto.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497


In [59]:
#--quality check the data

df_toronto[df_toronto["PostalCode"].isnull()]      #--nothing is null 
df_toronto[(df_toronto["Latitude"].isnull())|(df_toronto["Longitude"].isnull())]     #--nothing is null 

print("unique postcode: ", df_toronto["PostalCode"].unique().shape[0],
      "unique Borough: ", df_toronto["Borough"].unique().shape[0],   
      "unique Neighborhood: ", df_toronto["Neighborhood"].unique().shape[0],
      "unique coordinate: ",(df_toronto["Latitude"].astype(str)+df_toronto["Longitude"].astype(str)).unique().shape[0])

#--only postcode is unique, so use postcode for area analysis?  since lat and lng 

unique postcode:  103 unique Borough:  10 unique Neighborhood:  99 unique coordinate:  103


#### get venue list from foursquare using function made from explore

In [266]:
#--use foursquare!

#print(client_id, client_srt, version, limit) 
lat = None
lng = None


#--create a function for explore area of a given neighborhood  

def explore_area(names, latitudes, longitudes, radius = 500):    #--this is just the name for neighborhood/borough
    
    venue_lis = []
    for name, lat, lng in zip(names, latitudes, longitudes):
    
        url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(client_id, client_srt, lat, lng, version, radius, limit)
    
        venue = requests.get(url).json()["response"]["groups"][0]["items"]
        
        venue_lis.append([(name, lat, lng,
                       v["venue"]["name"],    
                       v["venue"]["categories"][0]["name"],    
                       v["venue"]["location"]["lat"],
                       v["venue"]["location"]["lng"]) for v in venue])
    
    df_venue = pd.DataFrame([item for venue_lis in venue_lis for item in venue_lis])
    df_venue.columns = ["neighborhood", "neighborhood_lat", "neighborhood_lng",
                        "venue", "categories", "venue_lat","venue_lng"] 
   
    return(df_venue)



#--apply the function to list of Toronto postal code

toronto_venue = explore_area(df_toronto["PostalCode"],df_toronto["Latitude"],df_toronto["Longitude"])
toronto_venue.sort_values(by = "neighborhood", ascending = True, inplace = True)


In [208]:
#--investigate the data a bit 

toronto_venue
toronto_venue.groupby("neighborhood").count()

post_not_in = []
for neighbor in df_toronto["PostalCode"].unique():
     #   
    if neighbor in (toronto_venue["neighborhood"].unique()):
        None
    else:
        post_not_in.append(neighbor)
        
        
post_not_in
#--why these two post have nothings? 

radius = 500
lat_test = float(df_toronto[df_toronto["PostalCode"] == "M1X"]["Latitude"])
lng_test = float(df_toronto[df_toronto["PostalCode"] == "M1X"]["Longitude"])

#url_test="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(client_id, client_srt,lat_test, lng_test, version, radius, limit)
#requests.get(url_test).json()#["response"]

#--guess they have no data on foursquare: totalResults: 0


#### create dummy to group venues, and then get frequency

In [219]:
#--venue by category within each neighborhood

toronto_summary = pd.concat([toronto_venue[["neighborhood"]], pd.get_dummies(toronto_venue[["categories"]])], axis = 1)
#--dummy encode for each venue!

toronto_summary.columns = ["neighborhood"] + [col.split("_")[-1] for col in toronto_summary.columns[1:]]
#--change column names 

toronto_summary.head()

toronto_grouped = toronto_summary.groupby("neighborhood").mean().reset_index().sort_values(by = "neighborhood")  
#--this is pct of type of venue in the neighborhood.

toronto_grouped.head()


,neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [228]:
#--get top by frequency 

#--just do a quick test to see there is data 
toronto_grouped[toronto_grouped["neighborhood"] == "M1E"].T.reset_index()[1:].sort_values(by = 2, ascending = False)



#--functions:

def most_common (row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


def top_venue(top):
    columns = ["neighborhood"]

    for ind in np.arange(top):
        columns.append("top_{}".format(ind+1))
        
    df = pd.DataFrame(columns = columns)
    df["neighborhood"] = toronto_grouped["neighborhood"]
    
    for ind in np.arange(toronto_grouped.shape[0]):
        df.iloc[ind,1:] = most_common(toronto_grouped.iloc[ind, :], top) 
        
    return(df)

top_venue(5)


,neighborhood,top_1,top_2,top_3,top_4,top_5
0,M1B,Fast Food Restaurant,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store
1,M1C,Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
2,M1E,Mexican Restaurant,Electronics Store,Breakfast Spot,Bank,Rental Car Location
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
4,M1H,Hakka Restaurant,Gas Station,Bank,Fried Chicken Joint,Athletics & Sports
...,...,...,...,...,...,...
96,M9N,Park,Convenience Store,Yoga Studio,Doner Restaurant,Dessert Shop
97,M9P,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place
98,M9R,Park,Pizza Place,Sandwich Place,Bus Line,Discount Store
99,M9V,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pizza Place


#### create clusters and kmean, then add neighborhood data back in for plotting

In [231]:
#--create clusters and kmean

kclusters = 5

cluster_input = round(toronto_grouped.drop("neighborhood", axis = 1),2)

kmeans = KMeans(n_clusters = kclusters, random_state = 1).fit(cluster_input)
kmeans.labels_[:5]   #--remember this is an array! can't do head!


array([4, 0, 0, 0, 0])

In [259]:
#--consolidate data

cluster_temp = pd.concat([top_venue(5),pd.DataFrame(kmeans.labels_)], axis = 1)
#--they had the same order by neighborhood so should be good

toronto_cluster=cluster_temp.join(df_toronto.set_index("PostalCode"),on = "neighborhood")  
#--join df_toronto onto cluster because there were 2 post codes with no foursquare data!

toronto_cluster.rename(columns = {"neighborhood":"PostalCode", 0:"cluster"}, inplace = True)

toronto_cluster.head()

,PostalCode,top_1,top_2,top_3,top_4,top_5,cluster,Borough,Neighborhood,Latitude,Longitude
0,M1B,Fast Food Restaurant,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,4,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Mexican Restaurant,Electronics Store,Breakfast Spot,Bank,Rental Car Location,0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,0,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Hakka Restaurant,Gas Station,Bank,Fried Chicken Joint,Athletics & Sports,0,Scarborough,Cedarbrae,43.773136,-79.239476


In [263]:
#--plot it!!!

# create map
toronto_map_cluster = folium.Map(location = [can_lat, can_lng], zoom_start = 11)

cluster_colors = ["cadetblue", "white", "orange", "pink", "blue", "red","green"]
  

for lat, lng, neignbor, cluster in zip(toronto_cluster["Latitude"], toronto_cluster["Longitude"], 
                                    toronto_cluster["PostalCode"], toronto_cluster["cluster"]):
    label = folium.Popup("{},{}".format(neignbor,cluster), parse_html = True)
    folium.CircleMarker(location= [lat, lng], radius = 10, popup = label, 
                       color = "black", fill_color = cluster_colors[cluster],  
                        fill = True, fill_opacity = 0.9).add_to(toronto_map_cluster)

toronto_map_cluster
